In [1]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'gay',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI'
]

In [2]:
import json
import random

In [13]:
data = []

with open('/home/husein/ssd3/ctranslate2/function-call.translated.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        texts = []
        for no, d in enumerate(l['conversations']):
            
            if d['role'] == 'user':
                t = '<manusia>: '
                if random.random() > 0.5:
                    c = 'content'
                else:
                    c = 'content_ms'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content_ms'
                t = '<bot>: '

            if d[c] is None:
                break
            
            if 'functioncall' in d[c]:
                d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

            n = d[c]
            n = n.strip()
            if len(n) < 3:
                break
            t = t + n
            texts.append(t)
            
        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue
        
        data.append({
            'prompt_input': None,
            'input': '\n'.join(texts),
            'output': None,
            'function_call': l['function_call'],
        })

In [18]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [19]:
print(generate_and_tokenize_prompt(data[-4])['text'])

<s>
[FUNCTIONCALL]
{
    "name": "calculate_loan_interest",
    "description": "Calculate the interest on a loan",
    "parameters": {
        "type": "object",
        "properties": {
            "principal": {
                "type": "number",
                "description": "The principal amount of the loan"
            },
            "interest_rate": {
                "type": "number",
                "description": "The annual interest rate"
            },
            "duration": {
                "type": "integer",
                "description": "The duration of the loan in months"
            }
        },
        "required": [
            "principal",
            "interest_rate",
            "duration"
        ]
    }
}
[INST] Bolehkah anda membantu saya mengira faedah pinjaman? [/INST] Pasti! Saya boleh membantu anda dengan itu. Sila berikan saya maklumat yang diperlukan.</s> [INST] The principal amount is $5000, the annual interest rate is 5%, and the duration is 36 months. [/I

In [41]:
from tqdm import tqdm

for d in data:
    generate_and_tokenize_prompt(d)['text']

In [42]:
print(generate_and_tokenize_prompt(data[4])['text'])

<s>
[FUNCTIONCALL]
{
    "name": "calculate_median",
    "description": "Calculate the median of a list of numbers",
    "parameters": {
        "type": "object",
        "properties": {
            "numbers": {
                "type": "array",
                "items": {
                    "type": "number"
                },
                "description": "A list of numbers"
            }
        },
        "required": [
            "numbers"
        ]
    }
}
[INST] Hi, I have a list of numbers and I need to find the median. Can you help me with that? [/INST]Tentu saja, saya boleh membantu anda dengan itu. Sila berikan saya senarai nombor.</s> [INST] Nombornya ialah 5, 2, 9, 1, 7, 4, 6, 3, 8. [/INST]<functioncall> {"name": "calculate_median", "arguments": '{"numbers": [5, 2, 9, 1, 7, 4, 6, 3, 8]}'} 


<functioncall> {"median": 5}</s> 


In [20]:
print(generate_and_tokenize_prompt(data[1])['text'])

<s>
[FUNCTIONCALL]
{
    "name": "get_news_headlines",
    "description": "Get the latest news headlines",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "The country for which to fetch news"
            }
        },
        "required": [
            "country"
        ]
    }
}
[INST] Can you tell me the latest news headlines for the United States? [/INST] <functioncall> {"name": "get_news_headlines", "arguments": '{"country": "United States"}'}</s> [INST] Itu menarik. Bagaimana dengan berita di Perancis? [/INST] <functioncall> {"name": "get_news_headlines", "arguments": '{"country": "France"}'}</s> 


In [21]:
import json

with open('prepared-glaive-function-call.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')